In [ ]:
import pandas
import numpy as np
import ipywidgets
from IPython.display import display

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

In [ ]:
df2011 = pandas.read_csv('data/2011_Housing_Inventory_latlng_zip.csv')
df2012 = pandas.read_csv('data/2012_Housing_Inventory_latlng.csv')
df2013 = pandas.read_csv('data/2013_Housing_Inventory_latlng.csv')
df2014 = pandas.read_csv('data/2014_Housing_Inventory_latlng.csv')
df_map = {2011: df2011, 2012: df2012, 2013: df2013, 2014: df2014}

In [ ]:
def normalize_netunits(df):
    if 'STDADDRESS' in df:
        grouped = df.groupby('STDADDRESS')
    else:
        grouped = df.groupby('STDADD')
        
    return grouped.aggregate({'NETUNITS': np.sum, 'LAT_LNG': lambda x: next(iter(x))})

In [ ]:
def create_circles_for_df(m, df):
    circles = []
    normalized_df = df#normalize_netunits(df)
    
    try:
        target = normalized_df.AFF_TARGET
    except AttributeError:
        target = normalized_df.TARGET
    
    for netunit, latlng, affhsg in zip(normalized_df.NETUNITS, normalized_df.LAT_LNG, target):
        if latlng == '[]' or netunit == 0:
            continue

        latlng = latlng[1:-1].split(', ')
        lat = float(latlng[0])
        lng = float(latlng[1])

        #if netunit < 0:
        #    continue
        #color = {
        #    'VLI': '#0F0',
        #    'LI': '#080',
        #    'MOD': '#FF0',
        #    'SEC': '#00F',
        #}.get(affhsg, '#F00')
        #fill_color = color
        
        if netunit > 0:
            g = max(255 - 5*netunit, 40)
            color = "#{:0>2x}{:0>2x}{:0>2x}".format(0, g, 0)
            fill_color = color
        else:
            r = max(255 + 5*netunit, 40)
            color = "#{:0>2x}{:0>2x}{:0>2x}".format(r, 0, 0)
            fill_color = color

        c = Circle(location=(lat,lng), radius=50, color=color, fill_color=fill_color, opacity=1.0)
        circles.append(c)
        #m.add_layer(c)
        
    # add all circles at once. HACK!!
    for layer in circles:
        layer._map = m
    m.layers = tuple([l for l in m.layers] + circles)
    for layer in circles:
        layer.visible = True
        
    return circles

In [ ]:
circles = []
def on_slider_change(w):
    global circles
    m.layers = tuple([m.layers[0]])
    circles = create_circles_for_df(m, df_map[w['new']])
    
slider = ipywidgets.IntSlider(max=2014, min=2011)
slider.observe(on_slider_change, 'value')

In [ ]:
center = [37.76284413400416, -122.39490509033203]
zoom = 12
m = Map(default_tiles=TileLayer(opacity=1.0, url='https://tiles.wmflabs.org/bw-mapnik/{z}/{x}/{y}.png'), center=center, zoom=zoom)
create_circles_for_df(m, df2011)
display(m)
display(slider)